libraries in use 

In [114]:
import os 
import pandas as pd 
import numpy as np

file location constants 

In [115]:
rawDataPath = './data/unprocessed/'
erpSummaryPath ='./data/processed/'

generate number of students who passes or failes or need to do a supplementary 
includes their admission number 

In [116]:
def readErpSummary(file):
    #read from sheet number 3 erp summary
    df = pd.read_excel(file,sheet_name=2)

    #rename top heading column
    df.rename(columns = {'CAT':'CA', 'UNIT CODE':'UNIT_CODE' }, inplace = True) 
    
    unit_code = df['UNIT_CODE'][0]

    #null exam and cat values extract 
    nan_ca_exam_values  =  df[df['EXAM'].isnull() & df['CA'].isnull() ]
    #Remove the null cat and exam rows 
    df.dropna(subset=['EXAM','CA'], inplace=True)
    
    #null cat or exam values extract 
    nan_ca_values       =  df[df['CA'].isnull()]
    nan_exam_values     =  df[df['EXAM'].isnull()]
    #Remove the null cat and exam rows 
    df.dropna(subset=['EXAM'], inplace=True)
    df.dropna(subset=['CA'], inplace=True)
    
    #all units with cat and exam results 
    df["TOTAL_100%"] = df["CA"] + df["EXAM"]

    distinction_list = df[df['TOTAL_100%'].between(69, 101)]
    credit_list = df[df['TOTAL_100%'].between(59, 69)]
    pass_list = df[df['TOTAL_100%'].between(49,59)]
    supplementary_list = df[df['TOTAL_100%'].between(39,50)]
    retake_list = df[df['TOTAL_100%'].between(-1,40)]

    # Customize the export settings
    custom_header = ['distinction', 'credit', 'pass', 'supplementary','retake']
    count_data = {
    'distinction': [len(distinction_list)],
    'credit':[ len(credit_list)],
    'pass': [len(pass_list)],
    'supplementary':[len(supplementary_list) + len(nan_exam_values) ],
    'retake': [len(retake_list) + len(nan_ca_values) ]
    } 
    
    
    statitstics = pd.DataFrame(count_data)
    
    #log to excel file info
    with pd.ExcelWriter(f'{erpSummaryPath}/{unit_code}.xlsx',mode='w') as excel_writer:
        statitstics.to_excel(excel_writer, sheet_name='Sheet1', index=False)
        
        if(len(retake_list) > 0):
            retake_students_info = pd.DataFrame(retake_list)
            retake_students_info.to_excel(excel_writer, sheet_name='Sheet2', index=False)
        
        if(len(supplementary_list) > 0):
            supplementary_Students_info = pd.DataFrame(supplementary_list)
            supplementary_Students_info.to_excel(excel_writer, sheet_name='Sheet3', index=False)
    
    print(statitstics)

List the raw data files containing the results 
Generate a summary based on the file 

In [ ]:
def list_files_recursive(path):
    for root, dirs, files in os.walk(path):
        for file in files:
            print(file)
            #process the summary 
            readErpSummary(os.path.join(root, file))


list_files_recursive(rawDataPath)